In [32]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [36]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()



In [37]:
train_images[0].shape

(28, 28)

In [38]:
#generate new axis as required by convolution layer
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [39]:
train_images[0].shape

(28, 28, 1)

In [42]:
model = models.Sequential()

### Extract Features using Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


### Fully Connected Classification Head
model.add(layers.Flatten())
model.add(layers.Dense(5000, activation='relu'))
model.add(layers.Dense(10))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5000)             

In [43]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 90s 2ms/sample - loss: 0.1141 - accuracy: 0.9647 - val_loss: 0.0413 - val_accuracy: 0.9874
Epoch 2/10
 4256/60000 [=>............................] - ETA: 59s - loss: 0.0377 - accuracy: 0.9886

KeyboardInterrupt: 

In [ ]:
Epoch 1/10
60000/60000 [==============================] - 51s 850us/sample - loss: 0.1479 - accuracy: 0.9541 - val_loss: 0.0392 - val_accuracy: 0.9883

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

# Predict using Handwritten image

In [ ]:
plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(tf.concat([train_images[i],train_images[i],train_images[i]], 2), cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
file = tf.io.read_file('8.jpg')
img = tf.image.decode_jpeg(file, channels=1)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, [28, 28])
#Invert so the Background is Black as in other imgs.
img = (img-1)*-1

plt.imshow(tf.concat([img,img,img], 2))
model.predict(img[tf.newaxis, ...])

# Warum Padding bei Max_Pool?
Pixel am Rand würden sonst evtl. nicht erfasst werden
-> Konnte in den TF-Docs nicht finden, dass die Randpixel durch den parameter 'SAME' gespiegelt werden. Ist bei "MAX-Pooling" wsl. auch egal ob gespiegelt oder 0, falls Wertebereich nicht negativ ist.

Anmerkung: Padding bei Convolution aus anderen Gründen... Convolution würde auch das Bild verkleinern. Will man nicht immer.. Außerdem würden Merkmale am Rand weniger oft von Filtern erfasst werden.


# Feed Forward & Backpropagation

## Feedforward
Ein oder merhere Datensätze werden von input bis zum Output-Layer gereicht. In einer Netzwerkarchitektur, wo der Datenfluss auschließlich vom Input zum Output (über die Zwischenschichten) passiert, spricht man von einem Feed Forward Network.

Im Falle des Backpropagation Algorithmus werden die Ergebnisse der zwischenschichten / Zwischenneuronen gespeichert, da Sie während des Backward pass benötigt werden.

## Backpropagation
Ist der Algorithmus um heruaszufinden, wie die Parameter im Netz geändert werden sollen um die Cost-Function zu minimieren.

Backpropagation besteht aus einem `forward pass`, der `Fehlerermittlung` und dem `reverse / backward pass`. 

Im `forward pass` werden die Ausgaben jeder einzelnes Neurons von der Eingabe bis zur Ausgabeschicht für einen Input ermittelt. Ein Input ist üblicherweise ein "Batch" aus mehreren Inputdatensäten.

Während der Fehlerermittlung: Die Fehler der Ausgabeeinheiten werden bestimmt, indem der Ausgabevektor (Durchpropagierter Input) des Netzes mit den Zielwerten verglichen wird.

`Backward pass`: In diesem Schritt werden die Gewichtungen der Verbindungen im Netzwerk angepasst. Die ermittelten Fehler breiten sich rückwärts von der Ausgabe- bis zur Eingabeschicht aus. Stärkere Gewichtungen werden stärker angepasst. Das Gradientenabstiegsverfahren bestimmt dabei, in welchem Maße die Gewichte verändert werden ("Große Steigung wird viel angepasst").

## 5.3 Auswirkung der Aktivierungsfunktionen

Die Gewichte und der Bias eines Neurons werden zusammen addiert und anschließend durch eine Aktivierungsfunktion in ein "Aktivierungspotential" umgewandelt. Dieses ist dann wiederrum die Eingabe eines Neurons aus einer Folgeschicht.

Auswirkungen?
1. Die Wahl der Aktivierungsfunktion beeinflusst die Performance während des Trainings. ReLU ist beispielsweise perfomanter wie z.B. Sigmoid
2. Die Zufällige Initialisierung der Neuronen hängt von der Wahl der Aktivierungsfunktiona ab. Stichwort Vanishing/Exploding Gradients.

## 5.4 Netz ohne Aktivierungsfunktion

Kombinationen von linearen Funktionen ergibt wieder eine lineare Funktion. So können nur "lineare Probleme" gelöst werden. Soll eine nichtlineare Funktion approximiert werden, müssen nicht lineare Funktionen her.

Custom Activation Function implementieren und ausprobieren.

## 5.5 & 5.6 Gradienten Abstiegsverfahren

Wird verwendet um die Parameter im Netzwerk stückweise anzupassen. Die Gewichte werden aktualisiert, indem der Gradient der Cost-Function multipliziert mit der Lernrate verwendet wird.

Alternative Methoden sind beispielsweise:
- Momentum: Verwendet vergange Gradienten um einen Momentum Vektor zu erstellen. 
- Adam, Adagrad, AdaMax, RMSProp


## 5.6 Gradienten Abstiegsverfahren 2



## 5.7

() um ein Objekt der Klasse zu instanzieren. Der Lernraten-parameter muss nicht mitgegeben werden. Default ist 0.01

## 5.9 Pooling ?
- Wird verwendet um die Parameter des Netzwerks zu reduzieren. Das hat mehrere Vorteile: Overfitting, Memory Usage und Computrational load.

`layers.MaxPooling2D((2, 2)` fügt ein Max Pooling Layer hinzu. 2x2 Bildbereiche werden zusammengefasst, indem nur der Maximale Wert übernommen wird. Standardmäßig ist die Stride-Size die gleiche wie die Kernel Size. Es gibt also keine Überlagerungen. Padding ist standardsmäßig "valid", also wird nicht gepadded.

## 5.10 ?

# 5.11

Das Modell ist overfittet, wenn es Trainingsdaten auswendig gelernt hat und nicht gut auf ungesehene Daten generalisiert.
Es gibt viele verschiedene Ursachen für Overfitting.
1. Modell zu groß - zu viele Parameter
2. Zu wenige Trainingsdaten

## 5.12 Dropout

Wird eingesetzt um Neuronen während des Trainings zufällig mit einer Wahrscheinlichkeit p zu deaktivieren. Üblicherweise wird es in den letzten 1-3 Layern (Meist vollvernetzt) im Netzwerk eingesetzt.

Dadurch wird das Netzwerk robuster und ist nicht so stark durch Overfitting gefährdet